# OvP project


## Imports and data loading

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import yake
# import rake
# from rake import RAKE
from rake_nltk import Rake
from keybert import KeyBERT
from langdetect import detect
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import spacy
import keyword_spacy
from keyword_spacy import KeywordExtractor

# import openai
# from keybert.llm import OpenAI
# from keybert import KeyLLM

# nltk.download('stopwords')
# nltk.download('punkt_tab')
# nltk.download('punkt')
import transformers
from transformers import pipeline

In [2]:
# import data
data = pd.read_excel("raw_data.xlsx")

print(data.head(2))

        CURSUS             LANGE_NAAM_NL  \
0   ASB-ACT-EN  ACT - Leef zoals je wilt   
1  ASB-ACTLEEF  ACT - Leef zoals je wilt   

                                          DOCENT_ROL  \
0  Brasjen DOCENT; Bruinsma DOCENT; Buiks DOCENT;...   
1  Brasjen DOCENT; Bruinsma DOCENT; Buiks DOCENT;...   

  OSS_ADF_UTILITY.HTML_TO_TEXT(H.INHOUD)  
0                                    NaN  
1                                    NaN  


In [3]:

OG_data = data.copy()
small_data = data.iloc[29995:3000]

## Pipeline creation for specialty extraction

In [ ]:
# load model
extractor = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct")

# Define your extraction function
def extract_specialty(row):
    prompt = f"""
    You are an academic data extractor.

    From the following course metadata, identify the teacher's likely area of academic specialization.
    Focus on the academic field or topic expertise (max 5 words).
    The column 'DOCENT_ROL' has an unspecified amount of teachers in the format Name ROLE. If the ROLE is not docent, teacher, examinator of examiner, assume it is not their specialty and keep their specialty blank. Always include every teacher etc even if they have no specialty.
    If there is no course description or if the course description is nan, extract the teachers but make their specialty blank. 

    Return JSON with keys: name, specialty.
    

    Course code: {row['CURSUS']}
    Course name: {row['LANGE_NAAM_NL']}
    Teacher and role: {row['DOCENT_ROL']}
    Course description: {row['OSS_ADF_UTILITY.HTML_TO_TEXT(H.INHOUD)']}
    """

    try:
        result = extractor(prompt, max_new_tokens=100)[0]["generated_text"]
        # print(result)
        return result
    except Exception as e:
        print("\nFAIl")
        return str(e)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# test case
new_data = extract_specialty(data.iloc[2998])
print(new_data)

In [ ]:
# small datset
small_data["structured_info"] = small_data.apply(extract_specialty, axis=1)

small_data.to_excel("created_data/teachers_with_specialties_local.xlsx", index=True)


In [ ]:
print("done")